# 🏦 Investment Research with Compliance Plan Review

## Overview

This notebook demonstrates **interactive plan review** in Magentic workflows for financial services - enabling compliance oversight and modification of research plans before execution. This pattern is essential for:

1. **Regulatory Compliance**: Ensure research follows SEC/FINRA guidelines
2. **Risk Management**: Human approval for client-facing materials
3. **Quality Assurance**: Verify AI-generated research methodology
4. **Iterative Refinement**: Collaborate with AI to optimize research approach

> ⚠️ **Financial Services Disclaimer**: This notebook demonstrates AI agent workflows for educational purposes. In production, all investment research must be reviewed by compliance before distribution.

### Industry Use Case: Compliance-Reviewed Investment Research

A wealth management firm requires compliance review of AI-generated research plans before execution to ensure:
- Research methodology meets regulatory standards
- Appropriate disclosures are included
- Data sources are approved and documented
- Client suitability considerations are addressed

### Key Concepts

| Concept | Description |
|---------|-------------|
| **`.with_plan_review()`** | Enables human review of generated plans |
| **`MagenticPlanReviewRequest`** | Event type for plan review requests |
| **`event_data.approve()`** | Accept the proposed plan |
| **`event_data.revise(feedback)`** | Provide feedback to modify the plan |

### Plan Review Lifecycle

```
Investment Research Request
           ↓
Orchestrator Generates Research Plan
           ↓
Request Compliance Review (with_plan_review)
           ↓
┌─────────────────────────────────┐
│   Compliance Reviews Plan       │
│  ├── approve() - Accept plan    │
│  └── revise(feedback) - Modify  │
└─────────────────────────────────┘
           ↓
Resume with Compliance Decision
           ↓
Execute Plan → Final Research Report
```

### Review Response Options

| Method | Action | Industry Use Case |
|--------|--------|--------------|
| **`approve()`** | Accept plan, continue execution | Research plan meets compliance standards |
| **`revise(feedback)`** | Modify plan with feedback | Add disclosures or adjust methodology |

## Prerequisites

- OpenAI API key configured: `OPENAI_API_KEY` environment variable
- Agent Framework installed: `pip install agent-framework`

## 1️⃣ Setup and Imports

In [ ]:
import asyncio
import json
import logging
from typing import cast

import os
from dotenv import load_dotenv
from agent_framework import (
    AgentRunUpdateEvent,
    ChatAgent,
    ChatMessage,
    MagenticBuilder,
    MagenticPlanReviewRequest,
    RequestInfoEvent,
    WorkflowOutputEvent,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

# Load environment variables from .env file
load_dotenv('../../.env')

## 2️⃣ Create Specialized Financial Agents

Multi-agent setup for investment research:
- **MarketResearcherAgent**: Market data, sector analysis, news gathering
- **QuantAnalystAgent**: Data analysis, summarization, insights
- **MagenticManager**: Orchestrator that coordinates the workflow

In [ ]:
# Create Azure OpenAI chat client
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)
print("✅ Azure OpenAI Chat Client created")

# Create specialized financial agents
market_researcher = ChatAgent(
    name="MarketResearcherAgent",
    description="Specialist in market research, sector analysis, and financial news gathering",
    instructions=(
        "You are a Financial Market Researcher at a wealth management firm. "
        "You gather market data, sector trends, company news, and economic indicators. "
        "Focus on factual information without performing calculations. "
        "Always cite sources and note the date of any market data you reference."
    ),
    chat_client=chat_client,
)
print("✅ Market Researcher Agent created")

quant_analyst = ChatAgent(
    name="QuantAnalystAgent",
    description="Data analyst who processes and summarizes research findings",
    instructions=(
        "You are a Quantitative Analyst at a wealth management firm. "
        "You analyze findings and create summaries with risk metrics. "
        "Always show your calculation methodology and assumptions."
    ),
    chat_client=chat_client,
)
print("✅ Quant Analyst Agent created")

manager_agent = ChatAgent(
    name="MagenticManager",
    description="Orchestrator that coordinates the investment research workflow",
    instructions=(
        "You coordinate a team of financial analysts to complete investment research tasks efficiently. "
        "Ensure all research includes appropriate risk disclosures."
    ),
    chat_client=chat_client,
)
print("✅ Manager Agent created")

## 3️⃣ Build Workflow with Plan Review

Use `.with_plan_review()` to enable compliance review of generated plans before execution.

In [ ]:
print("\n🏦 Building Investment Research Workflow with Compliance Review...")

workflow = (
    MagenticBuilder()
    .participants([market_researcher, quant_analyst])
    .with_standard_manager(
        agent=manager_agent,
        max_round_count=10,
        max_stall_count=1,
        max_reset_count=2,
    )
    .with_plan_review()  # Enable human/compliance review of plans
    .build()
)

print("✅ Magentic workflow built with plan review enabled")

## 4️⃣ Define Investment Research Request

In [ ]:
# Client research request from a wealth advisor
research_request = (
    "Research the current state of sustainable investment funds (ESG) "
    "and summarize the findings for a client considering adding ESG exposure "
    "to their portfolio. Include recent performance trends and risk considerations."
)

print(f"📋 Research Request: {research_request}")

## 5️⃣ Run the Compliance-Reviewed Research Workflow

### ⚠️ IMPORTANT: How to Respond to Plan Review

When prompted for plan review, **an input box will appear at the top of VS Code**.

> **You MUST type your response in the input box, then press Enter.**  
> - Press **Enter with empty input** to **approve** the plan
> - Type **feedback text** then Enter to **revise** the plan with your feedback

### Example Responses:

| Input | Action |
|-------|--------|
| *(empty - just press Enter)* | Approve the plan as-is |
| `Add risk disclosures` | Revise plan with your feedback |
| `Include diversification recommendations` | Revise plan with your feedback |

In [ ]:
async def run_compliance_reviewed_research():
    """Run the Magentic investment research workflow with compliance plan review."""
    
    print("\n🚀 Starting research workflow (compliance review required)...")
    print("=" * 60)
    
    pending_request: RequestInfoEvent | None = None
    pending_responses: dict[str, object] | None = None
    output_event: WorkflowOutputEvent | None = None
    review_count = 0
    
    while not output_event:
        # First iteration uses run_stream, subsequent use send_responses_streaming
        if pending_responses is not None:
            stream = workflow.send_responses_streaming(pending_responses)
        else:
            stream = workflow.run_stream(research_request)
        
        last_message_id: str | None = None
        async for event in stream:
            if isinstance(event, AgentRunUpdateEvent):
                message_id = event.data.message_id
                if message_id != last_message_id:
                    if last_message_id is not None:
                        print("\n")
                    print(f"- {event.executor_id}:", end=" ", flush=True)
                    last_message_id = message_id
                print(event.data, end="", flush=True)
            
            elif isinstance(event, RequestInfoEvent) and event.request_type is MagenticPlanReviewRequest:
                pending_request = event
            
            elif isinstance(event, WorkflowOutputEvent):
                output_event = event
        
        pending_responses = None
        
        # Handle plan review request if any
        if pending_request is not None:
            review_count += 1
            event_data = cast(MagenticPlanReviewRequest, pending_request.data)
            
            print("\n\n" + "=" * 60)
            print(f"🏦 COMPLIANCE PLAN REVIEW #{review_count}")
            print("=" * 60)
            
            if event_data.current_progress is not None:
                print("\n📊 Current Progress Ledger:")
                print(json.dumps(event_data.current_progress.to_dict(), indent=2))
            
            print(f"\n📋 Proposed Plan:\n{event_data.plan.text}\n")
            print("-" * 60)
            print("👨‍💼 COMPLIANCE OFFICER: Review the plan above")
            print("-" * 60)
            
            # CLEAR PROMPT TEXT - shows options directly in the input dialog
            reply = await asyncio.get_event_loop().run_in_executor(
                None, 
                input, 
                "Enter APPROVE (or empty) to approve, or type feedback to revise: "
            )
            
            if reply.strip() == "" or reply.strip().upper() == "APPROVE":
                print("\n✅ Plan APPROVED by compliance.\n")
                pending_responses = {pending_request.request_id: event_data.approve()}
            else:
                print(f"\n📝 Plan REVISED with feedback: {reply}\n")
                pending_responses = {pending_request.request_id: event_data.revise(reply)}
            
            pending_request = None
    
    # Display final output
    print("\n" + "=" * 60)
    print("📊 INVESTMENT RESEARCH REPORT COMPLETE")
    print("=" * 60)
    
    output_messages = cast(list[ChatMessage], output_event.data)
    if output_messages:
        output = output_messages[-1].text
        print(output)
    
    print(f"\n✅ Completed with {review_count} compliance review(s)")
    print("\n" + "=" * 60)
    print("⚠️ DISCLAIMER: This is for demonstration purposes only.")
    print("   Actual investment research requires full compliance review.")

## 6️⃣ Run the Research Workflow

In [ ]:
await run_compliance_reviewed_research()

## 📝 Key Takeaways

### Compliance Plan Review for FSI

| Benefit | Description |
|---------|-------------|
| **Regulatory Compliance** | Ensure research meets SEC/FINRA guidelines |
| **Quality Assurance** | Verify methodology before execution |
| **Risk Management** | Human approval for client-facing materials |
| **Audit Trail** | Document compliance decisions |

### Industry Use Cases for Plan Review

| Use Case | Review Focus |
|----------|-------------|
| Investment Research | Methodology, disclosures, suitability |
| Client Communications | Regulatory compliance, accuracy |
| Trade Recommendations | Risk assessment, conflict checks |
| Financial Reporting | Data sources, calculation methods |

### Production Considerations

| Consideration | Recommendation |
|---------------|----------------|
| **Audit Trail** | Log all compliance decisions with timestamps |
| **Timeout Handling** | Auto-escalate if review not completed in SLA |
| **Role-Based Access** | Only authorized compliance officers can approve |
| **Multi-Level Approval** | Senior approval for high-value research |